# A Biological Example

Suppose we are observing exponential growth $\dot{y} = \theta y$ but we don't know $\theta$ and wish to estimate it. We could assume $\theta \sim {\cal{N}}(\mu, \sigma^2)$ and use something like Markov Chain Monte Carlo or Hamiltonian Monte Carlo and any observations to infer $\mu$ and $\sigma$. However, we might want to model that the further we go into the future, the less we know about $\theta$. We can write our system as as

$$
\begin{aligned}
\mathrm{d}y & = \theta y\mathrm{d}t \\
\mathrm{d}\theta & = \sigma\mathrm{d}W_t
\end{aligned}
$$

where $W_t$ is Brownian Motion.

# Fokker-Planck

$$
d \mathbf{X}_{t}=\boldsymbol{\mu}\left(\mathbf{X}_{t}, t\right) d t+\boldsymbol{\sigma}\left(\mathbf{X}_{t}, t\right) d \mathbf{W}_{t}
$$

$$
\frac{\partial}{\partial t} p(t, \mathbf{x})+\sum_{k=1}^{n} \frac{\partial}{\partial x_{k}}\left({\mu}_{k}(t, \mathbf{x}) p(t, \mathbf{x})\right)=\frac{1}{2} \sum_{j=1, k=1}^{n} \frac{\partial^{2}}{\partial x_{j} \partial x_{k}}\left[\left(\sigma(t, \mathbf{x}) \sigma^{T}(t, \mathbf{x})\right)_{j k} p(t, \mathbf{x})\right]
$$

For our particular system we have

$$
\frac{\partial}{\partial t} p(t, y, \theta)+\frac{\partial}{\partial y}\left({\mu}_{1}(t, y, \theta) p(t, y, \theta)\right)+\frac{\partial}{\partial \theta}\left({\mu}_{2}(t, y, \theta) p(t, y, \theta)\right)=\frac{1}{2}\left[\sigma_{y}^{2} \frac{\partial^{2}}{\partial y^{2}} p(t, y, \theta)+\sigma_{\theta}^{2} \frac{\partial^{2}}{\partial \theta^{2}} p(t, y, \theta)\right]
$$

And since $\mu_1 = \theta y$, $\mu_2 = 0$ and $\sigma_y = 0$ this further simplifies to

$$
\frac{\partial}{\partial t} p(t, y, \theta)+\frac{\partial}{\partial y}(\theta y p(t, y, \theta))=\sigma_{\theta}^{2} \frac{\partial^{2}}{\partial \theta^{2}} p(t, y, \theta)
$$

We can note two things:

* This is an advection / diffusion equation with two spatial variables ($y$ and $\theta$).
* If $\sigma_\theta = 0$ then this is a transport (advection?) equation.

$$
\frac{\partial}{\partial t} p(t, y, \theta)+\frac{\partial}{\partial y}(\theta y p(t, y, \theta))=0
$$

Notice that there is nothing stochastic about the biology but we
express our uncertainty about the parameter by making it a
time-varying stochastic variable which says the further we go into the
future the less certain we are about it.

We are going to turn this into a Fokker-Planck equation which we can
then solve using e.g. the method of lines. But before turning to
Fokker-Planck, let's show that we can indeed solve a diffusion
equation using the method of lines.

Let us solve the heat equation

$$
\frac{\partial u}{\partial t}=k_{x} \frac{\partial^{2} u}{\partial x^{2}}+k_{y} \frac{\partial^{2} u}{\partial y^{2}}+h
$$

with initial condition $u(0, x, y) = 0$ and stationary boundary conditions

$$
\frac{\partial u}{\partial t}(t, 0, y)=\frac{\partial u}{\partial t}(t, 1, y)=\frac{\partial u}{\partial t}(t, x, 0)=\frac{\partial u}{\partial t}(t, x, 1)=0
$$

and a periodic heat source

$$
h(x, y)=\sin (\pi x) \sin (2 \pi y)
$$

This has analytic solution

$$
u(t, x, y)=\frac{1-e^{-\left(k_{x}+4 k_{y}\right) \pi^{2} t}}{\left(k_{x}+4 k_{y}\right) \pi^{2}} \sin (\pi x) \sin (2 \pi y)
$$

The spatial derivatives are computed using second-order centered differences, with the data distributed over $n_x \times n_y$ points on a uniform spatial grid.

$$
u_{i\,j}(t) \triangleq u\left(t, x_{i}, y_{j}\right), \quad x_{i} \triangleq i \Delta x, \quad 0 \leq i \leq n_x-1, \quad  y_{j} \triangleq j \Delta y, \quad 0 \leq j \leq n_y-1
$$

$$
\begin{align}
u_{x x} &= \frac{u_{i+1\,j}-2 u_{i\,j}+u_{i-1\,j}}{\Delta x^{2}} \\
u_{y y} &= \frac{u_{i\,j+1}-2 u_{i\,j}+u_{i\,j-1}}{\Delta y^{2}}
\end{align}
$$

$$
\dot{u}_{i\, j} = \frac{k_x}{(\Delta x)^2}({u_{i+1\,j}-2 u_{i\,j}+u_{i-1\,j}})
                + \frac{k_y}{(\Delta y)^2}({u_{i\,j+1}-2 u_{i\,j}+u_{i\,j-1}})
                + h_{i\, j}
$$

We could try using [Naperian functors and APL-like programming in Haskell](https://www.cs.ox.ac.uk/people/jeremy.gibbons/publications/aplicative.pdf) via this [library](http://hackage.haskell.org/package/Naperian). But the performance is terrible (or it could be that the author's implementation was terrible). Moreover, applied mathematicans tend to think of everything as matrices and vectors. But flattening the above tensor operation into a matrix operation is not entirely trivial. Although the Haskell Ecosystem's support for symbolic mathematics is very rudimentary, we can use what there is to convince ourselves that we haven't made too many errors in the transcription.

In [1]:
{-# LANGUAGE DataKinds           #-}
{-# LANGUAGE OverloadedLists     #-}
{-# LANGUAGE ScopedTypeVariables #-}
{-# LANGUAGE FlexibleContexts    #-}
{-# LANGUAGE FlexibleInstances   #-}
{-# LANGUAGE MultiParamTypeClasses #-}
{-# LANGUAGE GADTs               #-}
{-# LANGUAGE TypeApplications    #-}
{-# LANGUAGE TypeOperators       #-}

import Data.Number.Symbolic
import qualified Data.Number.Symbolic as Sym
import Data.Proxy

import qualified Naperian as N
import qualified GHC.TypeLits as M
import           Data.Functor

import           Numeric.Sundials.ARKode.ODE
import           Numeric.LinearAlgebra

$$
\dot{u}_{i\, j} = \frac{k_x}{(\Delta x)^2}\sum_{k=0}^{n_x-1}\sum_{l=0}^{n_y-1}A_{i\,j\,k\,l} u_{k\,l}
                + \frac{k_y}{(\Delta y)^2}\sum_{k=0}^{n_x-1}\sum_{l=0}^{n_y-1}B_{i\,j\,k\,l} u_{k\,l}
                + h_{i\, j}
$$


$$
A_{i\, j\, l\, k} =
\begin{cases}
0,& \text{if } i = 0 \\
0,& \text{if } j = 0 \\
1,& \text{if } k = i-1 \text{ and } l = j \\
-2,& \text{if } k = i \text{ and } l = j \\
1,& \text{if } k = i+1 \text{ and } l = j \\
0,& \text{if } i = n_x - 1 \\
0,& \text{if } j = n_y - 1 \\
0,& \text{otherwise}
\end{cases}
$$


$$
B_{i\, j\, l\, k} =
\begin{cases}
0,& \text{if } i = 0 \\
0,& \text{if } j = 0 \\
1,& \text{if } k = i \text{ and } l = j - 1 \\
-2,& \text{if } k = i \text{ and } l = j \\
1,& \text{if } k = i \text{ and } l = j + 1 \\
0,& \text{if } i = n_x - 1 \\
0,& \text{if } i = n_y - 1 \\
0,& \text{otherwise}
\end{cases}
$$

In [2]:
preA :: forall b m n . (M.KnownNat m, M.KnownNat n, Num b) =>
        N.Hyper '[N.Vector n, N.Vector m, N.Vector n, N.Vector m] b
preA = N.Prism $ N.Prism $ N.Prism $ N.Prism $ N.Scalar $
      N.viota @m <&> (\(N.Fin x) ->
      N.viota @n <&> (\(N.Fin w) ->
      N.viota @m <&> (\(N.Fin v) ->
      N.viota @n <&> (\(N.Fin u) ->
      (f m n x w v u)))))
        where
          m = fromIntegral $ M.natVal (undefined :: Proxy m)
          n = fromIntegral $ M.natVal (undefined :: Proxy n)
          f p q i j k l | i == 0               = 0
                        | j == 0               = 0
                        | i == p - 1           = 0
                        | j == q - 1           = 0
                        | k == i - 1 && l == j = 1
                        | k == i     && l == j = -2
                        | k == i + 1 && l == j = 1
                        | otherwise            = 0

In [32]:
a :: forall a m n . (M.KnownNat m, M.KnownNat n, Floating a, Eq a) =>
      N.Hyper '[N.Vector n, N.Vector m, N.Vector n, N.Vector m] (Sym a)
a = N.binary (*) (N.Scalar $ var "a") preA

In [4]:
preB :: forall b m n . (M.KnownNat m, M.KnownNat n, Num b) =>
        N.Hyper '[N.Vector n, N.Vector m, N.Vector n, N.Vector m] b
preB = N.Prism $ N.Prism $ N.Prism $ N.Prism $ N.Scalar $
      N.viota @m <&> (\(N.Fin x) ->
      N.viota @n <&> (\(N.Fin w) ->
      N.viota @m <&> (\(N.Fin v) ->
      N.viota @n <&> (\(N.Fin u) ->
      (f m n x w v u)))))
        where
          m = fromIntegral $ M.natVal (undefined :: Proxy m)
          n = fromIntegral $ M.natVal (undefined :: Proxy n)
          f :: Int -> Int -> Int -> Int -> Int -> Int -> b
          f p q i j k l | i == 0                   = 0
                        | j == 0                   = 0
                        | i == p - 1               = 0
                        | j == q - 1               = 0
                        | k == i     && l == j - 1 = 1
                        | k == i     && l == j     = -2
                        | k == i     && l == j + 1 = 1
                        | otherwise                = 0

In [33]:
b :: forall a m n . (M.KnownNat m, M.KnownNat n, Floating a, Eq a) =>
           N.Hyper '[N.Vector n, N.Vector m, N.Vector n, N.Vector m] (Sym a)
b = N.binary (*) (N.Scalar $ var "b") preB

In [34]:
preFoo = N.binary (+) (a @Double @3 @4) (b @Double @3 @4)

In [35]:
h :: forall m n a . (M.KnownNat m, M.KnownNat n, Floating a) =>
                     N.Hyper '[N.Vector m, N.Vector n] (Sym a)
h = N.Prism $ N.Prism $ N.Scalar $
     N.viota @n <&> (\(N.Fin x) ->
     N.viota @m <&> (\(N.Fin w) ->
     var ("u_{" ++ show x ++ "," ++ show w ++ "}")))

In [36]:
postFoo = N.binary (*) preFoo (h @4 @3)

In [37]:
postFoo

<<<<0.0,0.0,0.0,0.0>,<0.0,0.0,0.0,0.0>,<0.0,0.0,0.0,0.0>>,<<0.0,0.0,0.0,0.0>,<0.0,0.0,0.0,0.0>,<0.0,0.0,0.0,0.0>>,<<0.0,0.0,0.0,0.0>,<0.0,0.0,0.0,0.0>,<0.0,0.0,0.0,0.0>>,<<0.0,0.0,0.0,0.0>,<0.0,0.0,0.0,0.0>,<0.0,0.0,0.0,0.0>>>,<<<0.0,0.0,0.0,0.0>,<0.0,0.0,0.0,0.0>,<0.0,0.0,0.0,0.0>>,<<0.0,c_2*u_{0,1},0.0,0.0>,<c_1*u_{1,0},((-2.0)*c_2+(-2.0)*c_1)*u_{1,1},c_1*u_{1,2},0.0>,<0.0,c_2*u_{2,1},0.0,0.0>>,<<0.0,0.0,c_2*u_{0,2},0.0>,<0.0,c_1*u_{1,1},((-2.0)*c_2+(-2.0)*c_1)*u_{1,2},c_1*u_{1,3}>,<0.0,0.0,c_2*u_{2,2},0.0>>,<<0.0,0.0,0.0,0.0>,<0.0,0.0,0.0,0.0>,<0.0,0.0,0.0,0.0>>>,<<<0.0,0.0,0.0,0.0>,<0.0,0.0,0.0,0.0>,<0.0,0.0,0.0,0.0>>,<<0.0,0.0,0.0,0.0>,<0.0,0.0,0.0,0.0>,<0.0,0.0,0.0,0.0>>,<<0.0,0.0,0.0,0.0>,<0.0,0.0,0.0,0.0>,<0.0,0.0,0.0,0.0>>,<<0.0,0.0,0.0,0.0>,<0.0,0.0,0.0,0.0>,<0.0,0.0,0.0,0.0>>>>

In [38]:
:t postFoo

postFoo :: Hyper '[Vector 4, Vector 3, Vector 4, Vector 3] (Sym Double)

In [42]:
(h @4 @3)

<<u_{0,0},u_{0,1},u_{0,2},u_{0,3}>,<u_{1,0},u_{1,1},u_{1,2},u_{1,3}>,<u_{2,0},u_{2,1},u_{2,2},u_{2,3}>>

In [41]:
N.foldrH (+) 0 $ N.foldrH (+) 0 postFoo

<<0.0,0.0,0.0,0.0>,<0.0,c_2*u_{0,1}+c_1*u_{1,0}+((-2.0)*c_2+(-2.0)*c_1)*u_{1,1}+c_1*u_{1,2}+c_2*u_{2,1},c_2*u_{0,2}+c_1*u_{1,1}+((-2.0)*c_2+(-2.0)*c_1)*u_{1,2}+c_1*u_{1,3}+c_2*u_{2,2},0.0>,<0.0,0.0,0.0,0.0>>

In [39]:
mapM_ (putStrLn . ("u_{0,0} &= " ++) . (++ " \\\\") . show) $
  fmap (sum . N.elements . N.Prism . N.Prism . N.Scalar) $
  N.elements $ N.crystal $ N.crystal postFoo

u_{0,0} &= 0.0 \\
u_{0,0} &= 0.0 \\
u_{0,0} &= 0.0 \\
u_{0,0} &= 0.0 \\
u_{0,0} &= 0.0 \\
u_{0,0} &= c_2*u_{0,1}+c_1*u_{1,0}+((-2.0)*c_2+(-2.0)*c_1)*u_{1,1}+c_1*u_{1,2}+c_2*u_{2,1} \\
u_{0,0} &= c_2*u_{0,2}+c_1*u_{1,1}+((-2.0)*c_2+(-2.0)*c_1)*u_{1,2}+c_1*u_{1,3}+c_2*u_{2,2} \\
u_{0,0} &= 0.0 \\
u_{0,0} &= 0.0 \\
u_{0,0} &= 0.0 \\
u_{0,0} &= 0.0 \\
u_{0,0} &= 0.0 \\

In [31]:
(h @4 @3)

<<u_{0,0},u_{0,1},u_{0,2},u_{0,3}>,<u_{1,0},u_{1,1},u_{1,2},u_{1,3}>,<u_{2,0},u_{2,1},u_{2,2},u_{2,3}>>

$$
\begin{aligned}
u_{0,0} &= 0.0 \\
u_{0,1} &= 0.0 \\
u_{0,2} &= 0.0 \\
u_{0,3} &= 0.0 \\
u_{1,0} &= 0.0 \\
u_{1,1} &= a*u_{0,1}+b*u_{1,0}+((-2.0)*a+(-2.0)*b)*u_{1,1}+b*u_{1,2}+a*u_{2,1} \\
u_{1,2} &= a*u_{0,2}+b*u_{1,1}+((-2.0)*a+(-2.0)*b)*u_{1,2}+b*u_{1,3}+a*u_{2,2} \\
u_{1,3} &= 0.0 \\
u_{2,0} &= 0.0 \\
u_{2,1} &= 0.0 \\
u_{2,2} &= 0.0 \\
u_{2,3} &= 0.0 \\
\end{aligned}
$$

In [13]:
fmap (N.elements . N.Prism . N.Prism . N.Scalar) $ N.elements $ N.crystal $ N.crystal preFoo

[[0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0],[0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0],[0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0],[0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0],[0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0],[0.0,c2,0.0,0.0,c1,(-2.0)*c2+(-2.0)*c1,c1,0.0,0.0,c2,0.0,0.0],[0.0,0.0,c2,0.0,0.0,c1,(-2.0)*c2+(-2.0)*c1,c1,0.0,0.0,c2,0.0],[0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0],[0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0],[0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0],[0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0],[0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0]]

In [60]:
postFoo

: 

In [35]:
foo = fmap (N.elements . N.Prism . N.Prism . N.Scalar) $
      N.elements $ N.crystal $ N.crystal $
      N.binary (+) (a @Double @3 @4) (b @Double @3 @4)

In [36]:
:t foo

foo :: [[Sym Double]]

In [38]:
mapM_ (putStrLn . show) $ fmap (N.elements . N.Prism . N.Prism . N.Scalar) $
  N.elements $ N.crystal $ N.crystal $
  N.binary (+) (a @Double @3 @4) (b @Double @3 @4)

[0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0]
[0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0]
[0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0]
[0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0]
[0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0]
[0.0,c2,0.0,0.0,c1,(-2.0)*c2+(-2.0)*c1,c1,0.0,0.0,c2,0.0,0.0]
[0.0,0.0,c2,0.0,0.0,c1,(-2.0)*c2+(-2.0)*c1,c1,0.0,0.0,c2,0.0]
[0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0]
[0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0]
[0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0]
[0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0]
[0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0]

In [51]:
h @3 @4

<<u_{0,0},u_{0,1},u_{0,2}>,<u_{1,0},u_{1,1},u_{1,2}>,<u_{2,0},u_{2,1},u_{2,2}>,<u_{3,0},u_{3,1},u_{3,2}>>

In [52]:
:t h @3 @4

h @3 @4 :: forall a. Floating a => Hyper '[Vector 3, Vector 4] (Sym a)

In [40]:
bar = [var $ (\(x,y) -> "u_{" ++ show x ++ "," ++ show y ++ "}") (x,y) | y <- [1..3], x <- [1..4]] :: [Sym Double]

In [41]:
:t bar

bar :: [Sym Double]

In [43]:
bar

[u_{1,1},u_{2,1},u_{3,1},u_{4,1},u_{1,2},u_{2,2},u_{3,2},u_{4,2},u_{1,3},u_{2,3},u_{3,3},u_{4,3}]

In [44]:
mapM_ (putStrLn . show . sum) $ map (\r -> zipWith (*) r bar) foo

Line 1: Fuse mapM_/map
Found:
mapM_ (putStrLn . show . sum) $ map (\ r -> zipWith (*) r bar) foo
Why not:
mapM_ ((putStrLn . show . sum) . (\ r -> zipWith (*) r bar)) fooLine 1: Use print
Found:
putStrLn . show . sum
Why not:
print . sum

0.0
0.0
0.0
0.0
0.0
c2*u_{2,1}+c1*u_{1,2}+((-2.0)*c2+(-2.0)*c1)*u_{2,2}+c1*u_{3,2}+c2*u_{2,3}
c2*u_{3,1}+c1*u_{2,2}+((-2.0)*c2+(-2.0)*c1)*u_{3,2}+c1*u_{4,2}+c2*u_{3,3}
0.0
0.0
0.0
0.0
0.0

$$
\begin{aligned}
u_{11} &= 0.0 \\
u_{12} &= 0.0 \\
u_{13} &= 0.0 \\
u_{14} &= 0.0 \\
u_{21} &= 0.0 \\
u_{22} &= c2*u_{2,1}+c1*u_{1,2}+((-2.0)*c2+(-2.0)*c1)*u_{2,2}+c1*u_{3,2}+c2*u_{2,3} \\
u_{23} &= c2*u_{3,1}+c1*u_{2,2}+((-2.0)*c2+(-2.0)*c1)*u_{3,2}+c1*u_{4,2}+c2*u_{3,3} \\
u_{24} &= 0.0 \\
u_{31} &= 0.0 \\
u_{32} &= 0.0 \\
u_{33} &= 0.0 \\
u_{34} &= 0.0
\end{aligned}
$$

Spatial mesh size:

In [15]:
nx, ny :: Int
nx = 30
ny = 60

Heat conductivity coefficients:

In [16]:
kx, ky :: Floating a => a
kx = 0.5
ky = 0.75

x and y mesh spacing:

In [ ]:
dx :: Floating a => a
dx = 1 / (fromIntegral nx - 1)
dy :: Floating a => a
dy = 1 / (fromIntegral ny - 1)

In [17]:
c1, c2 :: Floating a => a
c1 = kx/dx/dx
c2 = ky/dy/dy

In [21]:
bNum :: forall a m n . (M.KnownNat m, M.KnownNat n, Floating a) =>
        N.Hyper '[N.Vector n, N.Vector m, N.Vector n, N.Vector m] a
bNum = N.binary (*) (N.Scalar c1) preB

In [22]:
aNum :: forall a m n . (M.KnownNat m, M.KnownNat n, Floating a) =>
        N.Hyper '[N.Vector n, N.Vector m, N.Vector n, N.Vector m] a
aNum = N.binary (*) (N.Scalar c2) preA

In [32]:
bigA :: Matrix Double
bigA = fromLists $
       fmap (N.elements . N.Prism . N.Prism . N.Scalar) $
       N.elements $ N.crystal $ N.crystal $ N.binary (+)
       (aNum @Double @3 @4) (bNum @Double @3 @4)

In [33]:
bigA

(12><12)
 [ 0.0,   0.0, 0.0,   0.0,     0.0,     0.0,     0.0,     0.0,     0.0,     0.0,     0.0,     0.0
 , 0.0,   0.0, 0.0,   0.0,     0.0,     0.0,     0.0,     0.0,     0.0,     0.0,     0.0,     0.0
 , 0.0,   0.0, 0.0,   0.0,     0.0,     0.0,     0.0,     0.0,     0.0,     0.0,     0.0,     0.0
 , 0.0,   0.0, 0.0,   0.0,     0.0,     0.0,     0.0,     0.0,     0.0,     0.0,     0.0,     0.0
 , 0.0,   0.0, 0.0,   0.0,     0.0,     0.0,     0.0,     0.0,     0.0,     0.0,     0.0,     0.0
 , 0.0, 420.5, 0.0,   0.0, 2610.75, -6062.5, 2610.75,     0.0,     0.0,   420.5,     0.0,     0.0
 , 0.0,   0.0, 0.0,   0.0,     0.0,     0.0,     0.0,     0.0,     0.0,     0.0,     0.0,     0.0
 , 0.0,   0.0, 0.0, 420.5,     0.0,     0.0, 2610.75, -6062.5,     0.0,     0.0,     0.0,   420.5
 , 0.0,   0.0, 0.0,   0.0,     0.0,     0.0,     0.0,     0.0,     0.0,     0.0,     0.0,     0.0
 , 0.0,   0.0, 0.0,   0.0,     0.0,   420.5,     0.0,     0.0, 2610.75, -6062.5, 2610.75,     0.0
 , 0.0,   0

In [1]:
{-# LANGUAGE DataKinds #-}
{-# LANGUAGE OverloadedLists       #-}
{-# LANGUAGE ScopedTypeVariables #-}
{-# LANGUAGE FlexibleContexts    #-}
{-# LANGUAGE GADTs               #-}

In [2]:
import Data.Maybe
import Data.Number.Symbolic
import Data.Proxy

import qualified Naperian as N
import qualified Data.Foldable as F
import           Control.Applicative ( liftA2 )
import qualified GHC.TypeLits as M

import           Numeric.Sundials.ARKode.ODE
import           Numeric.LinearAlgebra

In [3]:
x1, a, x2 :: Double
x1 = 0
a = 1.0
x2 = a

In [4]:
y1, y2 :: Double
y1 = 0.0
y2 = 1.0

In [5]:
bigT :: Double
bigT = 1000.0

In [6]:
n :: Int
n = 2

In [7]:
dx :: Double
dx = a / (fromIntegral n + 1)

In [8]:
dy :: Double
dy = a / (fromIntegral n + 1)

In [9]:
beta, s :: Double
beta = 1.0e-5
s = beta / (dx * dx)

Heat conductivity coefficients

In [10]:
kx, ky :: Double
kx = 0.5
ky = 0.75

In [11]:
c1, c2 :: Double
c1 = kx/dx/dx
c2 = ky/dy/dy

We want to turn this into a matrix equation so that we can use `hmatrix-sundials`

In [12]:
bigAB :: Matrix Double
bigAB = assoc (n * n, n * n) 0.0 [((i, j), f (i, j)) | i <- [0 .. n * n - 1]
                                                      , j <- [i - n, i,  i + n]
                                                      , j `elem` [0 .. n * n -1]]
  where
    f (i, j) | i     == j = (-2.0) * c1
             | i - n == j = 1.0    * c1
             | i + n == j = 1.0    * c1
             | otherwise = error $ show (i, j)

bigAA :: Matrix Double
bigAA = diagBlock (replicate n bigA)
  where
    bigA :: Matrix Double
    bigA = assoc (n, n) 0.0 [((i, j), f (i, j)) | i <- [0 .. n - 1]
                                                , j <- [i-1..i+1]
                                                , j `elem` [0..n-1]]
      where
        f (i, j) | i     == j = (-2.0) * c2
                 | i - 1 == j = 1.0    * c2
                 | i + 1 == j = 1.0    * c2

bigAA :: Matrix Double
bigAA = bigAB + bigAA

In [13]:
bigAB

(4><4)
 [ -9.0,  0.0,  4.5,  0.0
 ,  0.0, -9.0,  0.0,  4.5
 ,  4.5,  0.0, -9.0,  0.0
 ,  0.0,  4.5,  0.0, -9.0 ]

In [14]:
bigAA

(4><4)
 [ -13.5,  6.75,   0.0,   0.0
 ,  6.75, -13.5,   0.0,   0.0
 ,   0.0,   0.0, -13.5,  6.75
 ,   0.0,   0.0,  6.75, -13.5 ]

$$
h(x, y)=\sin (\pi x) \sin (2 \pi y)
$$

In [15]:
n

2

In [16]:
bigZZ1 :: Matrix Double
bigZZ1 = assoc (m * m, m * m) 0.0 [((i, j), f (i, j)) | i <- [0 .. m * m - 1]
                                                      , j <- [0 .. m * m - 1]]
  where
    m = n + 2
    f (i, j) | i     == 0     = 0.0
             | j     == 0     = 0.0
             | i     == j     = (-2.0) * c1
             | i - n == j     = 1.0    * c1
             | i + n == j     = 1.0    * c1
             | i     == n + 1 = 0.0
             | j     == n + 1 = 0.0
             | otherwise      = 0.0


In [17]:
bigZZ1

(16><16)
 [ 0.0,  0.0,  0.0,  0.0,  0.0,  0.0,  0.0,  0.0,  0.0,  0.0,  0.0,  0.0,  0.0,  0.0,  0.0,  0.0
 , 0.0, -9.0,  0.0,  4.5,  0.0,  0.0,  0.0,  0.0,  0.0,  0.0,  0.0,  0.0,  0.0,  0.0,  0.0,  0.0
 , 0.0,  0.0, -9.0,  0.0,  4.5,  0.0,  0.0,  0.0,  0.0,  0.0,  0.0,  0.0,  0.0,  0.0,  0.0,  0.0
 , 0.0,  4.5,  0.0, -9.0,  0.0,  4.5,  0.0,  0.0,  0.0,  0.0,  0.0,  0.0,  0.0,  0.0,  0.0,  0.0
 , 0.0,  0.0,  4.5,  0.0, -9.0,  0.0,  4.5,  0.0,  0.0,  0.0,  0.0,  0.0,  0.0,  0.0,  0.0,  0.0
 , 0.0,  0.0,  0.0,  4.5,  0.0, -9.0,  0.0,  4.5,  0.0,  0.0,  0.0,  0.0,  0.0,  0.0,  0.0,  0.0
 , 0.0,  0.0,  0.0,  0.0,  4.5,  0.0, -9.0,  0.0,  4.5,  0.0,  0.0,  0.0,  0.0,  0.0,  0.0,  0.0
 , 0.0,  0.0,  0.0,  0.0,  0.0,  4.5,  0.0, -9.0,  0.0,  4.5,  0.0,  0.0,  0.0,  0.0,  0.0,  0.0
 , 0.0,  0.0,  0.0,  0.0,  0.0,  0.0,  4.5,  0.0, -9.0,  0.0,  4.5,  0.0,  0.0,  0.0,  0.0,  0.0
 , 0.0,  0.0,  0.0,  0.0,  0.0,  0.0,  0.0,  4.5,  0.0, -9.0,  0.0,  4.5,  0.0,  0.0,  0.0,  0.0
 , 0.0,  0.0,  0.0,  

$$
\dot{u}_{i\, j} = \frac{k_x}{(\Delta x)^2}({u_{i+1\,j}-2 u_{i\,j}+u_{i-1\,j}})
                + \frac{k_y}{(\Delta y)^2}({u_{i\,j+1}-2 u_{i\,j}+u_{i\,j-1}})
$$

In [18]:
x :: forall m n . (M.KnownNat m, M.KnownNat n) => N.Vector n (N.Vector m (Sym Int))
x = (fromJust . N.fromList) $
    map (fromJust . N.fromList) ([[var $ (\(x,y) -> "A" ++ show x ++ "," ++ show y) (x,y) | y <- [1..m]] | x <- [1..n]] :: [[Sym Int]])
    where
      m = M.natVal (undefined :: Proxy m)
      n = M.natVal (undefined :: Proxy n)

In [19]:
u1 :: N.Hyper '[N.Vector 3, N.Vector 2] (Sym Int)
u1 = N.Prism $ N.Prism (N.Scalar x)

In [20]:
u1

<<A1,1,A1,2,A1,3>,<A2,1,A2,2,A2,3>>

In [21]:
y :: forall n . M.KnownNat n => N.Vector n (Sym Int)
y = (fromJust . N.fromList) $
    (map (var . ("v" ++) . show) [1..n ] :: [Sym Int])
    where
    n = M.natVal (undefined :: Proxy n)

In [22]:
u2 :: N.Hyper '[N.Vector 3] (Sym Int)
u2 = N.Prism (N.Scalar y)

In [23]:
N.innerH u1 u2

<A1,1*v1+A1,2*v2+A1,3*v3,A2,1*v1+A2,2*v2+A2,3*v3>